# 6.1 sed & awk

* mächtige UNIX-Tools zur Textmanipulation und Datenverarbeitung


* __sed Stream Editor zum Bearbeiten von Datenströmen:__
    - Adressierung (Bedingung für Zeilenauswahl)
    - RegExp, Gruppierung
    - Kommandos (Optionen): delete (d), substitute (s//) usw.


* __awk *little language / DSL (domain specific language)* zur Datenmanipulation:__
    - umfangreicher als sed, enthält Kontrollstrukturen, Stringmethoden, mathematische Grundfunktionen usw.
    - Verwendung zum Einlesen und Manipulieren etwa von tabularen Daten
    - Field Separator (Zeilen in Felder als Tokens splitten und bearbeiten)
    - Syntax: `PATTERN/Bedingung {ACTION/Anweisungsblock}`
    - RegExp, Gruppierung
    - awk Arrays

## ed
| Name | ed - Zeilenorientierter Texteditor |
|:---|:---|
|Überblick| ed \[OPTION\]... \[FILES\]... |
| Beschreibung | Interaktiver Standardeditor (heuzutage überholt). |
| | Ließt von `stdin`, falls FILE in FILES `-` ist. |
| Wichtige Optionen: | |
| -p, --prompt=STRING | Verwende String für den interaktiven Prompt (Standardwert=?) |
| -s, --quiet, --silent | Unterdrückt die Ausgabe von Diagnosemeldungen |
| Zeilenadressierung: | |
| . | Aktuelle Zeile |
| 1 | Erste Zeile |
| 0 | Zeile vor der ersten |
| \$ | Letzte Zeile |
| n | Die `n`-te Zeile aus der Sequenz \[0,\$\] |
| /re/ | Die nexte Zeile die auf das Muster des regulären Ausdrucks `re` passt |
| ?re? | Die vorherige Zeile die auf das Muster des regulären Ausdrucks `re` passt |
| a1,a2 | Die Zeilen von `a1` bis `a2` (inklusive) |
| Befehle: |
| (.,.)p | Gibt die Zeilen in der Adresssequenz aus |
| (.,.)l | Listet die Zeilen mit ihren Zeilennummern in der Adresssequenz aus |
| (.,.)d | L&ouml;scht die Zeilen in der Adresssequenz aus |
| (.)i | Fügt Text vor der aktuellen Zeile in den Puffer ein; |
| | die Zeilen werden von `stdin` gelesen bis `\n.\n` |
| (.,.)c | Ändert (Überschreibt) die Zeilen in der Adresssequenz im Puffer; |
| | die Zeilen werden von `stdin` gelesen bis `\n.\n` |
| (.,.)s/re/replacement/ | Ersetzt erstes Vorkommen `re` mit `replacement` |
| (.,.)s/re/replacement/g | Ersetzt alle Vorkommen `re` mit `replacement` |
| (.,.)s/re/replacement/n | Ersetzt `n`-tes Vorkommen `re` mit `replacement` |
| w file | Schreibt den Puffer nach `file` |
| q | Beended ed |




---
### Beispiel: ed-Format in diff


### diff
| Name | diff - Vergleicht Dateien zeilenweise |
|:---|:---|
|Überblick| diff \[OPTION\]... FILES |
| Beschreibung | Vergleicht die Dateien in FILES. |
| Wichtige Optionen: | |
| -u, -u NUM, --unififed\[=NUM\]| Ausgabe von NUM (Standardwert=3) unifizierten Zeilen |
| -c, -c NUM, --context\[=NUM\]| Ausgabe von NUM (Standardwert=3) kopierten Zeilen |
| -e, --ed | Ausgabe eines ed-Skripts |
| -y, --side-by-side | Ausgabe in Zwei Spalten |
| -i, --ignore-case | Ignoriert Groß- und Kleinschreibung in Änderungen |
| -b, --ignore-space-change | Ignoriert die Anzahl der Leerzeichen in Änderungen |
| -B, --ignore-blank-lines | Ignoriert Änderungen auf leeren Zeilen |
| -w, --ignore-all-space | Ignoriert Änderungen auf allen Leerzeichen |
|Rückgabewerte:| |
| 0 | Dateien sind gleich |
| 1 | Dateien sind unterschiedlich |
| 2 | Fehler |


In [10]:
%%bash
diff -u a.py b.py || true

--- a.py	2021-05-18 11:37:53.085488900 +0200
+++ b.py	2021-05-18 11:37:58.125488900 +0200
@@ -2,7 +2,8 @@
 import sys
 
 def main():
-    print(f"Hello {sys.argv[1]}!")
-    
+    for name in sys.argv[1:]:
+        print(f"Hello {name}!")
+
 if __name__ == "__main__":
     main()


#### diff-Ausgabe im ed-Format:

- Adressierung: Zeilen 5,6
- c: Ändere mit folgendem Input

In [109]:
%%bash
diff --ed a.py b.py || true

5,6c
    for name in sys.argv[1:]:
        print(f"Hello {name}!")

.


---
## sed
* Stream-Editor verarbeitet Eingabezeilen von `stdin` und schreibt Zeilen nach `stdout`
* sed verwendet ähnliche Befehlssyntax wie ed (sed basiert auf ed)
* wie ed-Befehle akzeptieren sed-Befehle Zeilenadressen und/oder Adresssequenzen
* __Zeilenadressen können Zeilennummern oder reguläre Ausdrücke sein__

| Name | sed - Stream-Editor zur Filterung und Modifizierung von Text |
|:---|:---|
|Überblick| ed \[OPTION\]... \[SCRIPT\] \[FILES\]... |
| Beschreibung | Automatischer Editor zur Filterung und Modifizierung von Text in
| | Ein- und Ausgabeströmen. Ähnlich zu `ed(1)` läuft aber nur einmal über den
| | Eingabetext. |
| Wichtige Optionen: | |
| -n, --quiet, --silent | Keine automatische Ausgabe der Eingabezeilen (*Pattern Space*) |
| -e, --expression script | Füge `script` den auszuführenden Befehlen hinzu |
| -f, --file file | Füge den Inhalt von `file` den auszuführenden Befehlen hinzu |
| -i, --in-place | Verändert die Dateien direkt |
| -E, --regexp-extended | Verwendet erweiterte reguläre Ausdrücke (EREs) |

### Adressen
* `1`: adressiert die erste Zeile
* `n`: adressiert die `n`-te Zeile
* `$`: adressiert die letzte Zeile
* `f~s`: adressiert jede `s`-te Zeile von `f` aus
* `/re/`: adressiert jede Zeile, die der reguläre Ausdruck `re` akzeptiert
* `a1,+n`: adressiert jede Zeile von `a1` bis `n` folgende Zeilen
* `a1,~n`: adressiert jede Zeile von `a1` bis zu einer Zeile die durch `n` teilbar ist
* Addressen können auch für mehrere Befehle definiert werden: `a1{cmd1 cmd2 ...}`

### Quit (.)q
* Beendet die Ausgabe von Zeilen und beendet das Skript
* akzeptiert eine optionale Adresszeile
 

In [14]:
%%bash
sed 8q wahlverwandschaften_ocr.txt | cat -n  # Ausgabe bis Zeile 8

     1	Die
     2	Wahlverwandtschaften.
     3	
     4	Ein Roman
     5	von
     6	Goethe.
     7	
     8	Erster Theil.


In [15]:
%%bash
sed /Goethe/q wahlverwandschaften_ocr.txt | cat -n # Ausgabe bis RegExp-Match

     1	Die
     2	Wahlverwandtschaften.
     3	
     4	Ein Roman
     5	von
     6	Goethe.


### Print (.,.)p
* gibt den aktuellen *Pattern Space* aus (aktuelle Zeile)
* akzeptiert Adresssequenzen
* meist in Verbindung mit `-n` Kommandozeilenoption (Unterdrückung automatische Zeilenausgabe)

In [16]:
%%bash
sed -n /laufen/p wahlverwandschaften_ocr.txt # Filterung von Zeilen (Adressen) mit 'laufen'

die zwar keine Fläche, doch fortlaufende frucht-
laufen lassen.


In [21]:
%%bash
sed -n /laufen/,+1p wahlverwandschaften_ocr.txt # Filterung von Zeilen mit 'laufen' + folgende Zeile

die zwar keine Fläche, doch fortlaufende frucht-
bare Rücken bildete. Dorf und Schloß hin-
laufen lassen.



### Delete (.,.)d
* löscht die addressierten Zeilen (Filterung)
* akzeptiert Adresssequenzen

In [17]:
%%bash
sed '1,/Erstes Kapitel/d' wahlverwandschaften_ocr.txt | sed 10q # Löschen von Zeile 1 bis 'Erstes Kapitel'


Eduard — so nennen wir einen reichen
Baron im besten Mannesalter — Eduard
hatte in seiner Baumschule die schönste Stun-
de eines Aprilnachmittags zugebracht, um
frisch erhaltene Pfropfreiser auf junge Stäm-
me zu bringen. Sein Geschäft war eben
vollendet; er legte die Geräthschaften in das
Futteral zusammen und betrachtete seine Ar-
beit mit Vergnügen, als der Gärtner hinzu-


In [78]:
%%bash
sed '/^$/d' wahlverwandschaften_ocr.txt | sed 10q # Löschen von leeren Zeilen

Die
Wahlverwandtschaften.
Ein Roman
von
Goethe.
Erster Theil.
Tübingen,
in der J. G. Cottaischen Buchhandlung.
1809.
Die


### Transliterate (.,.)y/set1/set2/
* ersetzt Vorkommen von Zeichen in `set1` durch korrespondierende Zeichen in `set2` (String-Replace)
* akzeptiert Adresssequenzen

In [42]:
%%bash
sed y/ae/ea/ wahlverwandschaften_ocr.txt | sed 10q # ersetzt a mit e und umgekehrt

Dia
Wehlvarwendtscheftan.

Ein Romen
von
Goatha.

Erstar Thail.

Tübingan,


In [66]:
%%bash
sed -n '/Goethe/p' wahlverwandschaften_ocr.txt | sed 'y/oethe/OETHE/'

GOETHE.


In [28]:
%%bash
#Kombination von Befehlen (nur in GNU sed!):
sed -n /Goethe/'{  
y/oethe/OETHE/ 
p }' wahlverwandschaften_ocr.txt

GOETHE.


### Substitute (.,.)s/re/repl/\[flags\]
* sucht und ersetzt den regulären Ausdruck `re` mit `repl`
* akzeptiert Adresssequenzen
* optionale `flags` parameter:
  * g ersetzt alle Vorkommen von `re` in einer Zeile (global)
  * n ersetzt nur das n-te Vorkommen von `re`
  * p gibt die Zeile aus (meist in Kombination mit `-n`)
  * w `file` schreibt die Zeile nach `file`
* die Begrenzungszeichen (`/`) können durch beliebige andere Zeichen ersetzt werden: `s!/usr/mail!/usr2/mail!`
* `&` referenziert das gesamte gefundene Suchmuster
* `\1...\9` referenzieren die entsprechenden Untergruppen im gefundenen Suchmuster


In [107]:
%%bash
sed -nE 's/[Ww]/_/p' wahlverwandschaften_ocr.txt | head -1

_ahlverwandtschaften.


In [106]:
%%bash
#mit global Option (g)
sed -nE 's/[Ww]/_/gp' wahlverwandschaften_ocr.txt | head -1

_ahlver_andtschaften.


In [48]:
%%bash
# Vertauschung von Bigrammen:
sed -nE -e 's/([^ ]+)\s+([^ ]+)\s*/\2 \1 [&] /gp' wahlverwandschaften_ocr.txt | head

Roman Ein [Ein Roman] 
Theil. Erster [Erster Theil.] 
der in [in der ] G. J. [J. G. ] Buchhandlung. Cottaischen [Cottaischen Buchhandlung.] 
Theil. Erster [Erster Theil.] 
1 I. [I. 1] 
Kapitel. Erstes [Erstes Kapitel.] 
— Eduard [Eduard — ] nennen so [so nennen ] einen wir [wir einen ] reichen
im Baron [Baron im ] Mannesalter besten [besten Mannesalter ] Eduard — [— Eduard] 
in hatte [hatte in ] Baumschule seiner [seiner Baumschule ] schönste die [die schönste ] Stun-
eines de [de eines ] zugebracht, Aprilnachmittags [Aprilnachmittags zugebracht, ] um


In [106]:
%%bash
# Auszeichnung von Zahlen im Text:
# '#' als Trennerzeichen (/ muss nicht escapt werden)
sed -E -e 's#([0-9]+)#<num>\1</num>#g' wahlverwandschaften_ocr.txt | grep num

<num>1809</num>.
I. <num>1</num>
I. <num>2</num>
<num>2</num> *
I. <num>3</num>
<num>3</num> *
I. <num>4</num>
<num>4</num> *
I. <num>5</num>
<num>5</num> *
I. <num>6</num>
<num>6</num> *


---
## awk
* bennant nach den Initialen der drei Ursprungsautoren:
  * Alfred V. Aho
  * Peter J. Weinberger
  * Brian Kernighan
* bietet eine `little language` zur Daten- bzw. Textmanipulation
* liegt irgendwo zwischen Shell-Skripten und echten Programmiersprachen

| Name | awk - Mustererkennungs- und Textmanipulationssprache |
|:---|:---|
|Überblick| awk \[-W option\] \[-F value\] \[-v var=value\] \[--\] 'program text' \[file\]... |
|         | awk \[-W option\] \[-F value\] \[-v var=value\] \[-f program-file\]... \[--\] \[file\]... |
| Beschreibung | Interpreter für die AWK Programmiersprache (Beschrieben in Aho, |
|              | Kerninghan and Weinberger, The AWK Programming Language, |
|              | Addison-Wesley Publishing, 1988). |
|              | Ein AWK Programm ist eine Sequenz von `pattern {actions...}` Paaren |
|              | und Funktionsdefinitionen. |
| Wichtige Optionen: | |
| -F value | Setzt den Feldseparator FS auf `value` |
| -f file | Programm wird von `file` gelesen |
| -v var=value | Setzt `var` auf `value` |
| -- | Markiert eindeutig das Ende der Kommandozeilenoptionen |


### awk Grundlagen
* Dateien bzw. `stdin` werden zeilenweise gelesen
* jede Zeile wird an einem Feldseparator aufgespalten
* die einzelnen Felder werden in den Variablen `$1, $2, ...` gespeichert
* in der Variable `$0` ist die gesamte Zeile gespeichert
* weitere vordefinierte Variablen:
  * `FS` Field-Separator (Standardwert=' ')
  * `OFS` Ausgabefeld-Separator (Standardwert=' ')
  * `NF` Anzahl der Felder in der aktuellen Zeile
  * `NR` aktuelle Zeilennummer
  * `FILENAME` aktueller Dateipfad
  * `SUBSEP` Separator für multidimensionale Arrays (siehe unten)
* für jede Zeile werden dann die `pattern {actions...}` Paare ausgeführt:
  * `pattern` definiert eine Bedingung (regulärer Ausdruck, logischer Test, usw.) für Zeilenadressierung
  * `actions...` werden nur dann ausgeführt wenn `pattern` auf die aktuelle Zeile passt
  * das spezielle `BEGIN {action...}` Paar wird einmal vor dem Lesen der Zeilen ausgeführt
  * das spezielle `END {action...}` Paar wird einmal nach dem Lesen der Zeilen ausgeführt
* alle Variablen sind global und existieren solange das Programm läuft
* Variablen werden standardmäßig mit 0 initialisiert
* Variablen sind Wahrheitswerte, Zahlen, Strings oder Arrays


In [127]:
%%bash
#Extraktion zweites Feld (mit Angabe Field-Separator FS):
echo "Max Mustermann, 9999, München" | awk 'BEGIN { FS = "," } ; { print $2 }'

 9999


In [128]:
%%bash
# (mit default FS:)
echo "Max Mustermann, 9999, München" | awk '{ print $2 }'

Mustermann,


In [84]:
%%bash
# Tokenisierung und Ausgabe Zeilen- und Tokennummer  (Felder):
# (ohne pattern > alle Zeilen)
awk '{ for (i = 1; i <= NF; i++) {cprint FILENAME ":" NR ":" i ": " $i}}' wahlverwandschaften_ocr.txt | sed 10q

wahlverwandschaften_ocr.txt:1:1: Die
wahlverwandschaften_ocr.txt:2:1: Wahlverwandtschaften.
wahlverwandschaften_ocr.txt:4:1: Ein
wahlverwandschaften_ocr.txt:4:2: Roman
wahlverwandschaften_ocr.txt:5:1: von
wahlverwandschaften_ocr.txt:6:1: Goethe.
wahlverwandschaften_ocr.txt:8:1: Erster
wahlverwandschaften_ocr.txt:8:2: Theil.
wahlverwandschaften_ocr.txt:10:1: Tübingen,
wahlverwandschaften_ocr.txt:11:1: in


In [89]:
%%bash
# pattern: leere Zeilen > Ersetzung mit <<BLANK>>, sonst: Ausgabe Zeile
awk '/^$/ {print "<<BLANK>>"} !/^$/ {print $0}' wahlverwandschaften_ocr.txt | sed 10q

Die
Wahlverwandtschaften.
<<BLANK>>
Ein Roman
von
Goethe.
<<BLANK>>
Erster Theil.
<<BLANK>>
Tübingen,


### Relationale und Boolsche Operatoren

| Operator | Beschreibung |
| :--- | :--- |
| `<`  | kleiner als |
| `>`  | größer als |
| `<=` | kleiner oder gleich |
| `>=` | größer oder gleich |
| `==` | gleich |
| `!=` | gleich |
| `~`  | Mustervergleich (regulärer Ausdruck) |
| `!~` | negierter Mustervergleich (regulärer Ausdruck) |
| `&&` | logisches und |
| <code>&#124;</code> | logisches oder |
| `!`  | Negation |


### Arithmetische Operatoren und Zuweisungen

| Operator | Beschreibung |
| :--- | :--- |
| `+`  | Addition |
| `-`  | Subtraktion |
| `*` | Multiplikation |
| `/` | Division |
| `^` | Potenz |
| `%` | Modulo |
| `++` | Inkrement |
| `--` | Dekrement |
| `=` | Zuweisung |
| `+=` | Weist das Ergebnis einer Addition zu |
| `-=` | Weist das Ergebnis einer Subtraktion zu |
| `*=` | Weist das Ergebnis einer Multiplikation zu |
| `/=` | Weist das Ergebnis einer Division zu |
| `%=` | Weist das Ergebnis einer Modulo-Operation zu |
| `^=` | Weist das Ergebnis einer Potenz-Operation zu |




In [86]:
%%bash
# Filterung von ungeraden Zeilen mit mehr als 8 Tokens (Feldern):
awk 'NF > 8 && NR%2 != 0 {print NR " " $0}' wahlverwandschaften_ocr.txt | sed 10q

57 rechts das Thal und man sieht über die rei-
153 dige habe ich gesorgt; auch drückt es ihn nicht
165 zu legen, oder noch weiter zu studiren, sich wei-
183 gemäß. Er soll nicht wirken; er soll sich auf-
217 Ich fühle nur zu sehr, daß mir ein Mann die-
239 mit Recht, weil sie zu thun, zu wirken beru-
273 Jahren sind, so bin ich als Frau wohl älter
275 ich dir nicht versagen, was du für dein einzi-
297 zu kommen, nur für dich allein zu leben; laß
345 dem er sich die Stirne rieb, bey alle dem,


In [81]:
%%bash
# Filterung von Zeilen, ob erstes Token (Feld = $1) dem REGEXP (für Artikel) entspricht:
awk '$1 ~ /^d[erias]+$/ {print NR " " $0}' wahlverwandschaften_ocr.txt | sed 10q

26 de eines Aprilnachmittags zugebracht, um
33 des Herrn ergetzte.
42 der Gärtner. Die Mooshütte wird heute
47 das Dorf, ein wenig rechter Hand die Kir-
75 der Pfad nach den neuen Anlagen in zwey
160 das ist eigentlich seine Qual. Das Vielfache,
180 diese verschiedenen Gelegenheiten, diese An-
250 die Berufung des Hauptmanns nicht so ganz
268 der Erinnerung, wir liebten die Erinnerung,
295 das Aeußere und was ins Ganze geht. Meine


### Schleifen, Bedingungen und eingebaute Funktionen

| Operator | Beschreibung |
| :--- | :--- |
| `while (condition) {actions...}`  | führt `actions` aus solange `condition` wahr ist |
| `do {actions...} while (condition)`  | führt `actions` aus solange `condition` wahr ist (mindestens einmal) |
| `for (init; condition; increment) {actions...}` | führt `actions` aus solange `condition` wahr ist |
| `for (i in arr) {actions...}` | iteriert über alle Indizes des Arrays `arr` und führt `actions` in jedem Durchlauf aus |
| `if (condition) {actions...}`| führt `actions` aus falls `condition` wahr ist |
| `if (condition) {actions1...} else {actions2...}`| führt `actions1` aus falls `condition` wahr ist; ansonsten wird `actions2` ausgeführt |
| `if (condition2) {actions1...} else if (condition2) {actions2...} ...`| führt `actions1` aus falls `condition` wahr ist; ansonsten wenn `condition2` wahr ist wird `actions2` ausgeführt |
| `cos(expr), sin(expr), exp(expr), log(expr), ...` | verschiedene arithmetische Funktionen |
| `lower(str)` | konvertiert `str` in Kleinschreibung |
| `index(str, sub)` | liefert den Index von `sub` im String `str` zurück |
| `length(str)` | liefert die Länge des Strings zurück |
| `split(str, arr, sep)` | trennt `str` an `sep` auf und speichert die Felder im array `arr` |
| `print ...` | Gibt Strings nach `stdout` aus |
| `printf ...` | Gibt formatierte Strings nach `stdout` aus |


In [116]:
%%bash
# Ausgabe der Wörter mit mehr als 10 Buchstaben:
awk '{for (i = 1; i <= NF; i++) {if (length($i) > 10){print $i}}}' wahlverwandschaften_ocr.txt | sed 10q

Wahlverwandtschaften.
Cottaischen
Buchhandlung.
Wahlverwandtschaften.
Mannesalter
Aprilnachmittags
zugebracht,
Pfropfreiser
Geräthschaften
betrachtete


### Arrays
* awk verfügt über sog. *assoziativen Arrays*
* einziger komplexer Datentyp
* mit Python-Dictionaries vergleichbar
* Schlüssel müssen Strings oder Zahlen sein
* setzen mit `arr[key] = val`
* Abfragen mit `x = arr[key]`
* Löschen von Einträgen mit `delete arr[key]`
* Iteration über die Schlüssel eines Arrays mit `for (key in arr) {...}`
* multidimensionale Arrays mit `,` möglich: `arr[0, 0] = x` (siehe `SUBSEP` oben)


### awk-Skripte
* mit der `-f` Option können auch awk-Skripte ausgeführt werden
* mit einer Shebang Zeile können die Skripte auch direkt ausgeführt werden
* Kommandozeilenargumente stehen in der `ARGV` Variable
* Die Laufzeitumgebung steht in der `ENVIRON` Variable
* Kommentare wie in Python oder Shell-Skripten mit `#`


In [129]:
%%bash
#Input:
echo -e "1 2 3\n4 5 6"

1 2 3
4 5 6


In [100]:
%%bash
#Skript zum Transponieren einer Matrix:
echo -e "1 2 3\n4 5 6" | awk -f transpose.awk

1 4
2 5
3 6


In [1]:
%%bash 
cat transpose.awk

#!/usr/bin/awk -f
{
    for (i = 1; i <= NF; i++) {
        mat[NR, i] = $i
    }
}

NF > max {
    max = NF
}

END {
    for (j = 1; j <= max; j++) {
        pre = ""
        for (i = 1; i <= NR; i++) {
            printf "%s%d", pre, mat[i, j]
            pre = " "
        }
        printf "\n"
    }
}

## Übungsaufgaben
### 1. Aufgabe `gres`
Schreiben Sie ein Skript `gres.bash`, das für einen regulären Ausdruck
RE (im ERE-Format), eine Ersezung REPL und einem Dateipfad FILE von
der Kommandozeile, diejenigen Zeilen in der Datei FILE ausgibt, auf
die RE passt. Dabei soll jedes Vorkommen von RE in den ausgegebenen
Zeilen durch REPL ersetzt werden.
```bash
$cat file.txt
Fuchs, du
hast die
Gans gestohlen.
./gres.bash '[A-Z]' '00' file.txt
00uchs, du
00ans gestohlen.
$
```

Wegen den übergebenen regulären Ausdrücken auf der Kommandozeile
können Fehler im Skript auftreten.  Welche Fehler können auftreten und
wie kann man diese Fehler beheben?

### 2. Aufgabe `dirsize`
Schreiben Sie ein Skript `dirsize.bash`, das für die übergebenen
Verzeichnispfade die Größe (in Kilobytes) des Verzeichnisses berechnet
(= Summe der Größe aller Dateien unterhalb des Verzeichnisses). Sie
können davon ausgehen das sich unter den Verzeichnissen nur Dateien
befinden. Beispiel:
```bash
$./dirsize.bash dir1 dir2
dir1: 10K
dir2: 100K
$
```

Wie kann man das Skript so anpassen, dass es auch rekursiv die Größe
von Unterverzeichnissen mit beachtet?

### 3. Aufgabe `bigrams.awk`

Schreiben Sie ein awk-Skript `bigrams.awk`, das einen Text von `stdin`
ließt, die Bigramme im Eingabetext berechnet und dann die
Bigrammfolgen mit ihren
[λ-geglätteten](https://cis-sp2021.github.io/06_naive_bayes.pdf)
relativen Häufigkeiten ausgibt. Die Variable λ soll dem Skript auf der
Kommandozeile übergeben werden (Standardwert=1).  Beispiel:

```bash
$ cat file.txt | tr '[:punct:]' ' ' | awk -f bigrams.awk 0.1
...
erste bigram 0.3
zweites bigram 0.1
...
$
```